In [1]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-20 19:11:13--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-20 19:11:13 (9.18 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [4]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-20 19:15:51--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.04s   

2024-06-20 19:15:51 (16.2 MB/s) - ‘documents.json’ saved [658332/658332]



In [1]:
import minsearch
import json
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

True

In [4]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [5]:
with open('documents.json', 'r') as f:
    doc_raws = json.load(f)

In [6]:
documents = []
for course_dict in doc_raws:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
index = minsearch.Index(text_fields = ['question', 'text', 'section'],
                         keyword_fields = ['course']
                        )
index.fit(documents)

In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    result  = index.search(query = query,
                 boost_dict = boost,
                 num_results = 5
                )
    return result
    
def build_prompt(query, search_result):

    context = ""
    for doc in search_result:
        context =context + f"section :{doc['section']} \nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
    
    prompt_template = """
    You are a course teaching assistant. Answer the given QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}\n
    CONTEXT: {context}
    """.strip()
    prompt = prompt_template.format(question = query, 
                                    context = context).strip()
    return prompt

def llm (prompt):
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages =[{'role':'user', 'content': prompt}]
    )
    return response.choices[0].message.content

In [9]:
def rag(query):
    results = search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)
    return answer

In [11]:
query = "i love this course but setting up the dev env is a problamatic for me?"
rag(query)

"It sounds like you are enthusiastic about the course but are struggling with setting up the development environment. Here's some guidance that might help you:\n\n1. **GitHub Repository**: Ensure you have created a GitHub account and cloned the course repo to your local machine. There is a video tutorial to assist you with this process: [Git for Everybody: How to Clone a Repository from GitHub](https://www.youtube.com/watch?v=2UnIBpNMXaQ).\n\n2. **Local Repository Management**: Having the course repo locally will help you access the instructors' code and contribute by making pull requests if needed. Additionally, creating your own repositories for your notes and work could be beneficial. There is a [tutorial from Atlassian](https://www.atlassian.com/git/tutorials/setting-up-a-repository) that can guide you through this setup.\n\n3. **.gitignore**: Make sure you use a `.gitignore` file to prevent large files like databases, .csv, and .gz files, as well as files containing passwords or k